# Read pdf to text

In [ ]:
# get filenames

In [3]:
# importing required modules
import PyPDF2

def pdfToText(filename):
    # creating a pdf file object
    pdfFileObj = open(filename, 'rb')
        
    # creating a pdf reader object
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    # printing number of pages in pdf file
    # print(pdfReader.numPages)
        
    # creating a page object
    pageObj = pdfReader.getPage(0)
        
    # extracting text from page
    text = pageObj.extractText()
        
    # closing the pdf file object
    pdfFileObj.close()

    # return the text
    return(text)

In [4]:
import pandas as pd
import re, itertools
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/weils/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/weils/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
# cleaning source https://hub.packtpub.com/clean-social-media-data-analysis-python/

# remove whitespaces
resume = resume.strip()

# remove urls
resume = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', resume)

# remove punctuation
resume = re.sub(r'[^\w\s]','',resume)

# remove html tags
# resume = re.sub('<[^<]+?>', '', resume)

# standardise words
resume = ''.join(''.join(s)[:2] for _, s in itertools.groupby(resume))

# split attached words
# resume = " ".join(re.findall('[A-Z][^A-Z]*', resume))

# convert text to lowercase
resume = resume.lower()

# stopword removal
resume = ' '.join([word for word in resume.split() if word not in (stopwords.words('english'))])

# stemming and lemmatization
tokens = nltk.word_tokenize(resume)

print(resume)
# print(tokens)


NameError: name 'resume' is not defined

# Count Vectoriser method
https://github.com/adsieg/text_similarity/blob/master/Different%20Embeddings%20%2B%20Cosine%20Similarity%20%2B%20HeatMap%20illustration.ipynb

In [6]:
def cosine_distance_countvectorizer_method(s1, s2):
    
    # sentences to list
    allsentences = [s1 , s2]
    
    # packages
    from sklearn.feature_extraction.text import CountVectorizer
    from scipy.spatial import distance
    
    # text to vector
    vectorizer = CountVectorizer()
    all_sentences_to_vector = vectorizer.fit_transform(allsentences)
    text_to_vector_v1 = all_sentences_to_vector.toarray()[0].tolist()
    text_to_vector_v2 = all_sentences_to_vector.toarray()[1].tolist()
    
    # distance of similarity
    cosine = distance.cosine(text_to_vector_v1, text_to_vector_v2)
    print('Similarity of two sentences are equal to ',round((1-cosine)*100,2),'%')
    return cosine

In [7]:
ss1 = 'The president greets the press in Chicago'
ss2 = 'Obama speaks to the media in Illinois'

In [8]:
cosine_distance_countvectorizer_method(ss1 , ss2)

Similarity of two sentences are equal to  37.8 %


0.6220355269907728

# GloVe Embedding

In [9]:
gloveFile = "glove.6B/glove.6B.300d.txt"
gloveMatrixLength = 300
import numpy as np

def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    with open(gloveFile, encoding="utf8" ) as f:
        content = f.readlines()
    model = {}
    for line in content:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model

In [10]:
import re
from nltk.corpus import stopwords
import pandas as pd
# from sklearn.metrics.pairwise import euclidean_distances

def preprocess(raw_text):
    # keep only words
    letters_only_text = re.sub("[^a-zA-Z]", " ", raw_text)
    # convert to lower case and split 
    words = letters_only_text.lower().split()
    # remove stopwords
    stopword_set = set(stopwords.words("english"))
    cleaned_words = list(set([w for w in words if w not in stopword_set]))
    return cleaned_words


def getVector(word):    
    try:
        wordVector = model[word]
    except KeyError:
        wordVector = np.zeros(gloveMatrixLength)
    return wordVector


def cosine_distance_between_two_words(word1, word2):
    import scipy
    return (1- scipy.spatial.distance.cosine(getVector(word1), getVector(word2)))


def calculate_heat_matrix_for_two_sentences(s1,s2):
    s1 = preprocess(s1)
    s2 = preprocess(s2)

    result_list = [[cosine_distance_between_two_words(word1, word2) for word2 in s2] for word1 in s1]
    result_df = pd.DataFrame(result_list)
    result_df.columns = s2
    result_df.index = s1
    return result_df


def cosine_distance_wordembedding_method(s1, s2):
    import scipy
    vector_1 = np.mean([getVector(word) for word in preprocess(s1)],axis=0)
    vector_2 = np.mean([getVector(word) for word in preprocess(s2)],axis=0)
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    return (round((1-cosine)*100,2))

def euclidean_distance_wordembedding_method(s1, s2):
    from scipy.spatial import distance
    vector_1 = np.mean([getVector(word) for word in preprocess(s1)],axis=0)
    vector_2 = np.mean([getVector(word) for word in preprocess(s2)],axis=0)
    euclidean = distance.euclidean(vector_1, vector_2)
    return (euclidean)


def heat_map_matrix_between_two_sentences(s1,s2):
    df = calculate_heat_matrix_for_two_sentences(s1,s2)
    import seaborn as sns
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(5,5)) 
    ax_blue = sns.heatmap(df, cmap="YlGnBu")
    # ax_red = sns.heatmap(df)
    cosine_distance_wordembedding_method(s1, s2)
    return ax_blue


## Load glove model

In [11]:
model = loadGloveModel(gloveFile)

Loading Glove Model
Done. 400000  words loaded!


## Test on local files (same folder as this ipynb)

In [12]:
CV_selected = pdfToText("Sample Field Support CV 1 (ML - Selected).pdf")
CV_rejected = pdfToText("Sample Field Support CV 2 (ABA - Rejected).pdf")
JD = pdfToText("Job Description - Field Support Engineer .pdf")

cosine_distance_wordembedding_method(CV_selected, JD)
cosine_distance_wordembedding_method(CV_rejected, JD)


85.68

## Get files from data folder

In [13]:
import os
filepath = "/Users/weils/Documents/term 7/01.400 Capstone/data/Field Support Engineer/CV/"
cv_files = os.listdir(filepath)
print(cv_files)

['Sample Field Support CV 4 (MSBI - Rejected).pdf', 'Sample Field Support CV 3 (GQ - Rejected).pdf', 'Sample Field Support CV 2 (ABA - Rejected).pdf', 'Sample Field Support CV 5 (LCG - Selected).pdf', 'Sample Field Support CV 1 (ML - Selected).pdf']


## Create for loop to run through all files

In [15]:
JD = pdfToText("/Users/weils/Documents/term 7/01.400 Capstone/data/Field Support Engineer/JD/Job Description - Field Support Engineer .pdf")

for cv_filename in cv_files:
    cv_text = pdfToText(filepath + cv_filename)

    cosine_similarity = cosine_distance_wordembedding_method(cv_text, JD)
    euclidean_distance = euclidean_distance_wordembedding_method(cv_text, JD)
    print("For", cv_filename, ", Cosine Similarity is", cosine_similarity, "and Euclidean Distance is", euclidean_distance)


For Sample Field Support CV 4 (MSBI - Rejected).pdf , Cosine Similarity is 85.77 and Euclidean Similarity is 1.1474418519510101
For Sample Field Support CV 3 (GQ - Rejected).pdf , Cosine Similarity is 9.89 and Euclidean Similarity is 3.815679108723644
For Sample Field Support CV 2 (ABA - Rejected).pdf , Cosine Similarity is 85.68 and Euclidean Similarity is 1.203572334927045
For Sample Field Support CV 5 (LCG - Selected).pdf , Cosine Similarity is 79.49 and Euclidean Similarity is 1.3594722297525004
For Sample Field Support CV 1 (ML - Selected).pdf , Cosine Similarity is 90.05 and Euclidean Similarity is 0.9379363274897352


## Try on other role

In [17]:
import os
filepath = "/Users/weils/Documents/term 7/01.400 Capstone/data/System Analyst/CV/"
cv_files = os.listdir(filepath)
# print(cv_files)

JD = pdfToText("/Users/weils/Documents/term 7/01.400 Capstone/data/System Analyst/JD/Job Description - System Analyst.pdf")

for cv_filename in cv_files:
    cv_text = pdfToText(filepath + cv_filename)

    cosine_similarity = cosine_distance_wordembedding_method(cv_text, JD)
    euclidean_distance = euclidean_distance_wordembedding_method(cv_text, JD)
    print("For", cv_filename, ", Cosine Similarity is", cosine_similarity, "and Euclidean Distance is", euclidean_distance)

For Sample System Analyst CV 3 (KA - Selected).pdf , Cosine Similarity is 87.77 and Euclidean Distance is 1.1577361762251361


ValueError: array must not contain infs or NaNs